# Standard Errors

## Linear Regression

In [133]:
import numpy as np

np.random.seed(42)

# Generate some linear data
x = np.linspace(0, 10, 100).reshape(-1, 1)  # Shape (100, 1)
y = 2 * x + 1 + np.random.randn(100, 1) * 0.5  # Shape (100, 1)

class LinearRegression:
    def __init__(self):
        self.slope = None
        self.intercept = None

    def fit(self, x, y):
        # Add a column of ones to X for the intercept
        X = np.hstack([np.ones((x.shape[0], 1)), x])  # Shape (100, 2)
        # Compute parameters using the normal equation
        params = np.linalg.inv(X.T @ X) @ X.T @ y  # Shape (2, 1)
        # Separate slope and intercept
        self.intercept = params[0, 0]  # Intercept is the first parameter
        self.slope = params[1, 0]  # Slope is the second parameter

    def calculate_se(self, x, y):
        # Add a column of ones to X for the intercept
        X = np.hstack([np.ones((x.shape[0], 1)), x])  # Shape (n, 2)
        
        # Calculate residuals
        residuals = y.flatten() - self.predict(x).flatten()
        
        # Residual Sum of Squares (RSS)
        rss = np.sum(residuals ** 2)
        
        # Degrees of freedom: n - p
        n, p = X.shape
        variance = rss / (n - p)
        
        # Variance-Covariance matrix of coefficients
        cov_matrix = variance * np.linalg.inv(X.T @ X)
        
        # Standard errors are the square root of the diagonal elements
        return np.sqrt(np.diag(cov_matrix))

    def predict(self, x):
        # Add a column of ones to X for the intercept
        X = np.hstack([np.ones((x.shape[0], 1)), x])  # Shape (n, 2)
        return X @ np.array([self.intercept, self.slope]).flatten()

# Instantiate and fit the model
model = LinearRegression()
model.fit(x, y)

# Predictions
predictions = model.predict(x)
standard_errors = model.calculate_se(x, y)

# Print the learned parameters
print(f"b0: {model.intercept:.4f}, b1: {model.slope:.4f}")
print(f"SE b0: {standard_errors[0]:.3f}, SE b1: {standard_errors[1]:.3f}")

b0: 0.9136, b1: 2.0069
SE b0: 0.091, SE b1: 0.016


In [144]:
from sklearn.linear_model import LinearRegression
import numpy as np

model_sk = LinearRegression()
model_sk.fit(x, y)

predictions = model_sk.predict(x)

def calculate_sk_st_errors(x, y):
   X = np.column_stack((np.ones(len(x)), x))
   rss = np.sum((y - model_sk.predict(x))**2)
   n, p = X.shape
   variance = rss / (n - p)
   cov_matrix = variance * np.linalg.inv(np.dot(X.T, X))
   se_b0 = np.sqrt(cov_matrix[0, 0])
   se_b1 = np.sqrt(cov_matrix[1, 1])
   return se_b0, se_b1

se_b0, se_b1 = calculate_sk_st_errors(x, y)
print(f"b0: {model_sk.intercept_}, b1: {model_sk.coef_}")
print(f"SE(b0): {se_b0:.3f}, SE(b1): {se_b1:.3f}")

b0: [0.91359357], b1: [[2.00689663]]
SE(b0): 0.091, SE(b1): 0.016


In [145]:
import statsmodels.api as sm

X = sm.add_constant(x)
model_sm = sm.OLS(y, X).fit()

print(model_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.647e+04
Date:                Mon, 13 Jan 2025   Prob (F-statistic):          5.37e-111
Time:                        12:24:24   Log-Likelihood:                -62.345
No. Observations:                 100   AIC:                             128.7
Df Residuals:                      98   BIC:                             133.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9136      0.091     10.094      0.0

## Logistic Regression

In [204]:
import numpy as np

# Generate random data
np.random.seed(42)

x = np.random.rand(100, 1)
y = 1 + 2 * x + 0.1 * np.random.randn(100, 1)
y = (y > 1.5).astype(int).flatten()

# Sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

class LogisticRegression:
    def __init__(self, max_iter=100):
        self.max_iter = max_iter
        self.weights = None
        self.epsilon = 1e-5

    def fit(self, x, y):
        # Add a column of ones to X for the intercept
        X = np.hstack([np.ones((x.shape[0], 1)), x])  # Shape (n, p+1)
        self.weights = np.zeros(X.shape[1])  # Initialize weights (p+1,)

        for iteration in range(self.max_iter):
            # Predictions (sigmoid function)
            predictions = sigmoid(X @ self.weights)

            # Gradient (score vector)
            score = X.T @ (y - predictions)

            # Diagonal of weights for Hessian
            W_diag = (predictions * (1 - predictions)).ravel()
            W = np.diag(W_diag)

            # Hessian (Information matrix)
            information = X.T @ W @ X

            # Update weights using Newton-Raphson method
            beta_new = self.weights + score @ np.linalg.inv(information)

            # Check for convergence
            if np.linalg.norm(beta_new - self.weights) < self.epsilon:
                self.weights = beta_new
                print(f"Converged in {iteration + 1} iterations.")
                break

            self.weights = beta_new

    def calculate_st_errors(self, x):
        # Predictions
        X = np.hstack([np.ones((x.shape[0], 1)), x])  # Shape (n, p+1)
        information = X.T @ np.diag((self.predict_proba(x) * (1 - self.predict_proba(x))).ravel()) @ X
        # Standard errors
        return np.sqrt(np.diag(np.linalg.inv(information)))

    def predict_proba(self, x):
        # Add a column of ones to X for the intercept
        X = np.hstack([np.ones((x.shape[0], 1)), x])  # Shape (n, p+1)
        # Return probabilities
        return sigmoid(X @ self.weights)

    def predict(self, x):
        # Convert probabilities to binary predictions
        return (self.predict_proba(x) >= 0.5).astype(int)

# Instantiate and fit the logistic regression model
model = LogisticRegression()
model.fit(x, y)

# Print learned weights
print(f"Weights: {model.weights}")

st_errors = model.calculate_st_errors(x)
print(f"Standard errors: {st_errors}")


Converged in 11 iterations.
Weights: [-10.1445658   42.48382473]
Standard errors: [ 3.21784182 13.21419489]


In [244]:
from sklearn.linear_model import LogisticRegression

model_sk = LogisticRegression(penalty=None, solver='newton-cg')
model_sk.fit(x, y)

def calculate_sk_st_errors(model, x):
   X = np.column_stack((np.ones(len(x)), x))
   predictions = model.predict_proba(x)[:, 1]
   W = np.diag(predictions * (1 - predictions))
   information = X.T @ W @ X
   return np.sqrt(np.diag(np.linalg.inv(information)))

print(f"b0: {model_sk.intercept_}, b1: {model_sk.coef_}")

se_b0, se_b1 = calculate_sk_st_errors(model_sk, x)
print(f"SE(b0): {se_b0:.3f}, SE(b1): {se_b1:.3f}")

b0: [-9.97027934], b1: [[41.73871733]]
SE(b0): 3.141, SE(b1): 12.884


In [245]:
import statsmodels.api as sm

X = sm.add_constant(x)

model_sm = sm.Logit(y, X).fit()
print(model_sm.summary())

Optimization terminated successfully.
         Current function value: 0.069751
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  100
Model:                          Logit   Df Residuals:                       98
Method:                           MLE   Df Model:                            1
Date:                Mon, 13 Jan 2025   Pseudo R-squ.:                  0.8842
Time:                        12:52:15   Log-Likelihood:                -6.9751
converged:                       True   LL-Null:                       -60.215
Covariance Type:            nonrobust   LLR p-value:                 5.787e-25
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.1446      3.218     -3.153      0.002     -16.451      -3.838
x1            42.4838     13